In [1]:
import zipfile
import geopandas as gpd
import requests

zip_file_path = 'tl_2020_24_tabblock20.zip'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall()

shapefile_path = 'tl_2020_24_tabblock20.shp'
block_gdf = gpd.read_file(shapefile_path)

In [2]:
url = "https://transit.land/api/v2/rest/feed_versions/8523f088e05dcd273b369dd4d65eb771c73c0a22/download?apikey=T2Jljq0EQqqe1JMfijkn3RdtVxoYpwuq"
response = requests.get(url)

if response.status_code == 200:
    with open("output.zip", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

File downloaded successfully.


In [3]:
import os

zip_file_path = 'output.zip'
extract_to_folder = 'transitland'
os.makedirs(extract_to_folder, exist_ok=True)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall(extract_to_folder)

In [4]:
import pandas as pd

In [5]:
#CODE FOR BLOCKS
compressed_csv_file_home = 'md_rac_S000_JT00_2020.csv.gz'
compressed_csv_file_work = 'md_wac_S000_JT00_2020.csv.gz'
gdf_filtered = block_gdf[block_gdf['COUNTYFP20'] == '510']

df_origins_blocks = pd.read_csv(compressed_csv_file_home)
df_destinations_blocks = pd.read_csv(compressed_csv_file_work)

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(str)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(str)

df_origins_blocks = df_origins_blocks[(df_origins_blocks['h_geocode'].str[:5] == '24510') & (df_origins_blocks['C000'] != 0)]
df_destinations_blocks = df_destinations_blocks[(df_destinations_blocks['w_geocode'].str[:5] == '24510') & (df_destinations_blocks['C000'] != 0)]

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(int)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(int)

gdf_filtered['GEOID20'] = gdf_filtered['GEOID20'].astype(int)
origins_merged_blocks = gdf_filtered.merge(df_origins_blocks, left_on = 'GEOID20', right_on = 'h_geocode')
destinations_merged_blocks = gdf_filtered.merge(df_destinations_blocks, left_on = 'GEOID20', right_on = 'w_geocode')

#origins_merged = origins_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'h_geocode', 'C000']]
#destinations_merged = destinations_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'w_geocode', 'C000']]

origins_merged_blocks['id'] = origins_merged_blocks.index
destinations_merged_blocks['id'] = destinations_merged_blocks.index

/Users/tommyli/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
from shapely.geometry import Point

# Convert latitude and longitude columns to float data types
origins_merged_blocks['INTPTLAT20'] = origins_merged_blocks['INTPTLAT20'].astype(float)
origins_merged_blocks['INTPTLON20'] = origins_merged_blocks['INTPTLON20'].astype(float)
origins_merged_blocks['geometry'] = origins_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

destinations_merged_blocks['INTPTLAT20'] = destinations_merged_blocks['INTPTLAT20'].astype(float)
destinations_merged_blocks['INTPTLON20'] = destinations_merged_blocks['INTPTLON20'].astype(float)
destinations_merged_blocks['geometry'] = destinations_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

In [7]:
accurate_times = pd.read_csv('bus_accurate_data.csv')
accurate_times['scheduled_visit_time'] = pd.to_datetime(accurate_times['scheduled_visit_time'])
accurate_times['date'] = accurate_times['scheduled_visit_time'].dt.date
accurate_times.rename(columns={'vehicle_stop_id': 'stop_id'}, inplace=True)
def filter_by_time(trip_data):
    hour = trip_data.reset_index()['scheduled_visit_time'][0].hour
    if (hour >= 7) and (hour < 10):
        return trip_data
    else:
        return None

accurate_times = accurate_times.groupby("trip_id").apply(filter_by_time).reset_index(drop=True)
accurate_times['scheduled_visit_time'] = accurate_times['scheduled_visit_time'].dt.strftime('%H:%M:%S')
observed_mask = accurate_times['observed_visit_time'].notna()
accurate_times.loc[observed_mask, ['observed_visit_time']] = pd.to_datetime(accurate_times['observed_visit_time'][observed_mask]).dt.strftime('%H:%M:%S')
unique_dates = accurate_times['date'].unique()
filtered_dfs = {}
for date in unique_dates:
    filtered_dfs[date] = accurate_times[accurate_times['date'] == date]

/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/819209817.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  accurate_times = pd.read_csv('bus_accurate_data.csv')


In [8]:
from datetime import datetime

gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')

for date, filtered_df in filtered_dfs.items():
    merged_df = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 
                                                   'scheduled_visit_time', 'trip_distance_traveled']], on=['trip_id', 'stop_id'], how="left")

    merged_df = merged_df[merged_df['scheduled_visit_time'].notna()]

    observed_mask = merged_df['observed_visit_time'].notna()
    merged_df.loc[observed_mask, ['arrival_time', 'departure_time']] = merged_df.loc[observed_mask, 'observed_visit_time']
    merged_df.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

    merged_df.drop(columns=['observed_visit_time'], inplace=True)

    merged_df.to_csv(f'stop_times/stop_times_{date}.txt', index=False)

In [9]:
# for date, filtered_df in filtered_dfs.items():
#     gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')
#     gtfs_stop_times = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 'scheduled_visit_time']], on=['trip_id', 'stop_id'], how = "left")

#     observed_mask = gtfs_stop_times['scheduled_visit_time'].notna()
#     gtfs_stop_times.loc[observed_mask, ['arrival_time', 'departure_time']] = gtfs_stop_times.loc[observed_mask, 'scheduled_visit_time']
#     gtfs_stop_times.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

#     gtfs_stop_times['arrival_time'] = pd.to_datetime(gtfs_stop_times['arrival_time'])
#     gtfs_stop_times['departure_time'] = pd.to_datetime(gtfs_stop_times['departure_time'])

#     gtfs_stop_times['arrival_time'] = gtfs_stop_times['arrival_time'].dt.strftime('%H:%M:%S')
#     gtfs_stop_times['departure_time'] = gtfs_stop_times['departure_time'].dt.strftime('%H:%M:%S')

#     gtfs_stop_times.drop(columns=['observed_visit_time'], inplace=True)

#     gtfs_stop_times.to_csv(f'transitland/stop_times_{date}.txt', index=False)

In [9]:
current_folder = os.getcwd()  # Get the current working directory (where your IPython Notebook is located)
stop_times_folder = os.path.join(current_folder, 'stop_times')

folder_path = stop_times_folder
file_paths = []

# Loop through the files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        file_paths.append(os.path.join(folder_path, file))

In [10]:
def to_time(time_string):
    return datetime.strptime(time_string, "%H:%M:%S")

def get_time_diff(before, after):
    return datetime.strptime(after, "%H:%M:%S") - datetime.strptime(before, "%H:%M:%S")

def get_time_diff_rev(after, before):
    return datetime.strptime(after, "%H:%M:%S") - datetime.strptime(before, "%H:%M:%S")

In [11]:
def fill_missing_times(trip_df):
    known_times = trip_df.dropna(subset=["arrival_time"])
    if (len(known_times) == len(trip_df)) or len(known_times) == 0:
        return trip_df
    
    # Iterate through known time points to calculate and apply estimated times
    for i in range(1, len(known_times)):
        prev_row = known_times.iloc[i - 1]
        next_row = known_times.iloc[i]
        mask = (trip_df["arrival_time"].isna()) & \
            (trip_df["trip_distance_traveled"] >= prev_row["trip_distance_traveled"]) & \
                (trip_df["trip_distance_traveled"] <= next_row["trip_distance_traveled"])
        if len(trip_df[mask]) > 0:
            # Calculate time difference and speed for this segment
            time_difference = get_time_diff(prev_row["arrival_time"], next_row["arrival_time"])
            distance_difference = next_row["trip_distance_traveled"] - prev_row["trip_distance_traveled"]
            speed = distance_difference / time_difference.total_seconds()
            
            # Calculate and update estimated arrival and departure times for missing rows within the segment
            trip_df.loc[mask, "arrival_time"] = to_time(prev_row["arrival_time"]) + \
                pd.to_timedelta((trip_df.loc[mask, "trip_distance_traveled"] - prev_row["trip_distance_traveled"]) / speed, unit="s")
            trip_df.loc[mask, "arrival_time"] = pd.to_datetime(trip_df['arrival_time'][mask]).dt.strftime('%H:%M:%S')
    
    known_times.reset_index(inplace= True)

    # Impute the first stop
    cur_row = known_times.iloc[0]
    mask = (trip_df["arrival_time"].isna()) & (trip_df["trip_distance_traveled"] <= cur_row["trip_distance_traveled"])
    if len(trip_df[mask]) > 0:
        
        trip_df.loc[mask, "arrival_time"] = \
            (to_time(cur_row['arrival_time']) - \
             trip_df["scheduled_visit_time"][mask].apply(get_time_diff, args = (cur_row["scheduled_visit_time"],))).dt.strftime('%H:%M:%S')

    # Impute the last stop
    cur_row = known_times.iloc[len(known_times) - 1]
    mask = (trip_df["arrival_time"].isna()) & (trip_df["trip_distance_traveled"] >= cur_row["trip_distance_traveled"])
    if len(trip_df[mask]) > 0:

        trip_df.loc[mask, "arrival_time"] = \
            (to_time(cur_row['arrival_time']) + \
             trip_df["scheduled_visit_time"][mask].apply(get_time_diff_rev, args = (cur_row["scheduled_visit_time"],))).dt.strftime('%H:%M:%S')
    

    trip_df['departure_time'] = trip_df['arrival_time']
    return trip_df

for file_path in file_paths:
    result_df = pd.read_csv(file_path)
    result_df = result_df.groupby("trip_id").apply(fill_missing_times)
    result_df.drop(columns=['scheduled_visit_time', 'trip_distance_traveled'], inplace=True)
    result_df.to_csv(file_path, index=False)  

/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/707016325.py:50: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result_df = result_df.groupby("trip_id").apply(fill_missing_times)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/707016325.py:50: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(.

In [12]:
import random
import os
import pandas as pd

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def replace_data(trip_id):
    file_rand = random.sample(file_paths, len(file_paths))
    for filePath in file_rand:
        df = load_data(filePath)
        trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
        if trip_id not in trip_ids_to_replace:
            return df.loc[df.trip_id == trip_id] 

updated_folder = "stop_times_updated"
os.makedirs(updated_folder, exist_ok=True)  

for file in file_paths:
    df = load_data(file)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    for trip_id in trip_ids_to_replace:
        new_data = replace_data(trip_id)
        df = df.loc[df.trip_id != trip_id]
        df = df.append(new_data)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    #df.to_csv(f'updated_{file}', index=False)
    updated_file_path = os.path.join(updated_folder, f'updated_{os.path.basename(file)}')
    df.to_csv(updated_file_path, index=False)
    #df.to_csv(os.path.join(folder_path, f'{file}'), index=False)

/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/1044719252.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/1044719252.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/1044719252.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_881/1044719252.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n

In [13]:
import os
import shutil
import zipfile
from io import StringIO
import pandas as pd

stop_times_folder = 'stop_times_updated'
transitland_folder = 'transitland'
zipfiles_folder = 'zipfiles'
trips_folder = 'transitland/trips.txt'


for filename in os.listdir(stop_times_folder)[:1]:
    times = pd.read_csv(f"{stop_times_folder}/{filename}")
    trips = pd.read_csv('trips_copy.txt')  # Load trip_content as DataFrame
    unique_trip_ids = times['trip_id'].unique()
    trips = trips[trips['trip_id'].isin(unique_trip_ids)]
    
    #routes = pd.read_csv(f"{transitland_folder}/routes.txt")
    #unique_route_ids = trips['route_id'].unique()
    #routes = routes[routes['route_id'].isin(unique_route_ids)]
    #directions = pd.read_csv(f"{transitland_folder}/directions.txt")
    #directions = directions[directions['route_id'].isin(unique_route_ids)]
    trips.to_csv(trips_folder, index=False)
    times.to_csv(f'{transitland_folder}/stop_times.txt', index=False)
    #routes.to_csv(f"{transitland_folder}/routes.txt", index=False)
    #directions.to_csv(f"{transitland_folder}/directions.txt", index=False)

    
    zip_filename = f'{os.path.splitext(filename)[0]}.zip'
    zip_path = os.path.join(zipfiles_folder, zip_filename)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(transitland_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, transitland_folder)
                zipf.write(file_path, arcname)
        

    print(f'Processed {filename}')


Processed updated_stop_times_2023-06-20.txt


In [25]:
import zipfile
import os

# Path to the zip file
zip_file_path = 'zipfiles/updated_stop_times_2023-03-06.zip'

# Directory where you want to extract the contents
extract_path = 'zipfiles_test'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_path)

print("Files extracted successfully!")


Files extracted successfully!


In [15]:
%%time
# Allow 6 GB memory
import sys
sys.argv.append(["--max-memory", "6G"])

import datetime
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode

# # Filepath to OSM data
osm_fp = "maryland-latest.osm.pbf"

CPU times: user 1.35 s, sys: 154 ms, total: 1.51 s
Wall time: 1.39 s


Running in non-interactive shell, SIGINT handler is replaced by shell
Signal Handlers:
SIGSEGV: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_ONSTACK|SA_RESTART|SA_SIGINFO
SIGBUS: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGFPE: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGPIPE: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGXFSZ: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGILL: [libjvm.dylib+0x53327c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGUSR2: [libjvm.dylib+0x5338bc], sa_mask[0]=00000000000000000000000000000000, sa_flags=SA_RESTART|SA_SIGINFO
SIGHUP: [libjvm.dylib+0x531f3c], sa_mask[0]=11111111011111110111111111111111, sa_flags=SA_RESTART|SA_SIGINFO
SIGINT: SIG_IGN, sa_mask[0

In [27]:
travel_time_matrix = []


zip_jan_19 = 'zipfiles/updated_stop_times_2023-01-19.zip'
zip_jan_20 = 'zipfiles/updated_stop_times_2023-01-20.zip'
zip_jan_26 = 'zipfiles/updated_stop_times_2023-01-26.zip'
zip_jan_27 = 'zipfiles/updated_stop_times_2023-01-27.zip'
transport_network = TransportNetwork(
    osm_fp,
    [
        zip_jan_19,
        zip_jan_20,
        zip_jan_26,
        zip_jan_27,
        
    ]
)

In [28]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origins_merged_blocks,
    destinations=destinations_merged_blocks,
    departure=datetime.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.BUS, LegMode.WALK],
    breakdown = True,
)
travel_time_matrix_blocks = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix.append(travel_time_matrix_blocks)

In [ ]:
travel_time_matrix_red = []


zip_jan_19 = 'zipfiles/updated_stop_times_2023-01-19.zip'
zip_jan_20 = 'zipfiles/updated_stop_times_2023-01-20.zip'
zip_jan_26 = 'zipfiles/updated_stop_times_2023-01-26.zip'
zip_jan_27 = 'zipfiles/updated_stop_times_2023-01-27.zip'
redline = 'redline.zip'
transport_network = TransportNetwork(
    osm_fp,
    [
        zip_jan_19,
        zip_jan_20,
        zip_jan_26,
        zip_jan_27,
        redline
        
    ]
)

In [ ]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origins_merged_blocks,
    destinations=destinations_merged_blocks,
    departure=datetime.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.BUS, LegMode.WALK],
    breakdown = True,
)
travel_time_matrix_blocks_red = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_red.append(travel_time_matrix_blocks)

In [24]:
travel_time_matrix[0].dropna()

,from_id,to_id,travel_time
0,0,0,0.0
3,0,3,67.0
12,0,12,9.0
15,0,15,73.0
21,0,21,120.0
...,...,...,...
3402,8291,3402,102.0
3404,8291,3404,63.0
3405,8291,3405,76.0
3407,8291,3407,61.0


In [21]:
import pandas as pd
import os
import urllib.request
import gzip
import shutil
from itertools import accumulate
import zipfile


def download_files(url, raw_path):
    zip_path, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_path, "r") as f:
        f.extractall(raw_path)

def get_lodes_file(raw_path, lodes_type, file_name):
    lodes_file = raw_path + file_name
    if not os.path.isfile(lodes_file):
        print("Downloading LODES data into " + lodes_file)
        url = f"https://lehd.ces.census.gov/data/lodes/LODES7/md/{lodes_type}/{file_name}.gz"
        urllib.request.urlretrieve(url, raw_path + "tmp.gz")
        with gzip.open(raw_path + "tmp.gz", "rb") as f_in:
            with open(lodes_file, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(raw_path + "tmp.gz")
    lodes_data = pd.read_csv(lodes_file)
    return lodes_data

def set_paths(prefix = "../"):
    processed_relpath = f"{prefix}processed_data/"
    raw_relpath = f"{prefix}raw_data/"

    processed_path = os.path.join(processed_relpath)
    raw_path = os.path.join(raw_relpath)
    return raw_path, processed_path

def replace_tracts(df):
    return df.replace(["24510180100", "24510180200"], "24510280600")

def restrict_to_Baltimore(df, col):
    tracts = pd.read_csv("../raw_data/Baltimore_tracts.csv")['tracts'].astype(str).tolist()
    return df[df[col].isin(tracts)]

def extract_tract_FIPS(df, col):
    return df[col].apply(lambda x: str(x)[-11:])

def FIPS_to_str(df, col):
    return df[col].apply(lambda x: str(x)[:11])


def cumulativeSum(lst):
    return list(accumulate(lst))

In [22]:
import geopandas as gpd
import numpy as np
import pandas as pd
from geopy.distance import geodesic
import time
import os
#from utils import *

#raw_path, processed_path = set_paths(prefix="")

def add_red_line(gtfs_folder, period = 8, speed = 20, earliest_hour = 7, latest_hour = 10):

    num_of_trips = int((latest_hour - earliest_hour) * 60 / period)

    stations = gpd.read_file("Baltimore Red Line- Stations.csv", crs=4326)

    segments = gpd.read_file("Baltimore Red Line- Alignment.csv", crs=4326)

    res = segments.apply(lambda x: [y for y in x['geometry'].coords], axis=1).loc[0]

    shapes = pd.DataFrame()
    shapes['shape_id'] = ["RED" for i in range(len(res))]
    shapes['shape_pt_lat'] = [x for _, x in res]
    shapes['shape_pt_lon'] = [y for y, _ in res]
    shapes['shape_pt_sequence'] = [i + 1 for i in range(len(res))]
    distances = []
    coords_1 = (res[0][1], res[0][0])
    # Compute distance traveled for the shape file
    for y,x in res:
        coords_2 = (x,y)
        distances.append(geodesic(coords_1, coords_2).miles)
        coords_1 = coords_2
    shapes['shape_dist_traveled'] = cumulativeSum(distances)

    stations['lon'] = stations.geometry.x
    stations['lat'] = stations.geometry.y

    stations = stations.sort_values(by=['lon'])

    longitudes = stations['lon'].to_list()
    latitudes = stations['lat'].to_list()

     # Compute distance traveled for the red line with respect to each station
     # The distances don't add up to 14 miles exactly, because the stations are not ON THE RAIL LINE
     # while we compute the distances using Euclidean distances
     # but we believe the discrepancy is negligible
    res = segments.apply(lambda x: [y for y in x['geometry'].coords if (y[0] < longitudes[-1] and y[0] > longitudes[0])], axis=1).loc[0]

    res = res + [(latitudes[-1], longitudes[-1])]

    distance = 0
    distances = []
    coords_1 = (latitudes[0], longitudes[0])
    n = len(res)
    point = 0

    for _, station in stations.iterrows():
        station_y = station['lon']
        station_x = station['lat']
        while point < n:
            y,x = res[point]
            point += 1
            if(y < station_y):
                coords_2 = (x, y)
                distance += geodesic(coords_1, coords_2).miles
                coords_1 = coords_2
            else:
                coords_2 = (station_x, station_y)
                distance += geodesic(coords_1, coords_2).miles
                distances.append(distance)
                coords_1 = (x, y)
                distance = geodesic(coords_2, coords_1).miles
                break

    # Create dataframe for stops
    stops = pd.DataFrame()
    stops['stop_id'] = list(stations.index)
    stops['stop_name'] = stations['name']
    stops['stop_lat'] = stations['lat']
    stops['stop_lon'] = stations['lon']

    # Create dataframe for trips
    trips = pd.DataFrame()

    trips['route_id'] = [99999 for i in range(num_of_trips * 2)]
    trips['service_id'] = 301
    trips['trip_id'] = [i + 1 for i in range(num_of_trips * 2)]
    trips['trip_headsign'] = ['Centers for Medicare and Medicaid Services' for i in range(num_of_trips)] + ['Canton' for i in range(num_of_trips)]
    trips['direction_id'] = [0 for i in range(num_of_trips)] + [1 for i in range(num_of_trips)]
    trips['shape_id'] = "RED"

    # Create dataframe for stop times
    stop_times = pd.DataFrame()

    def to_time_str(t):
        time_format = time.strftime("%H:%M:%S", time.gmtime(int(t)))
        return time_format

    start = earliest_hour*60*60

    arrival_time = []
    trip_id = []
    stop_id = []
    stop_sequence = []
    for i in range(num_of_trips):
        start_time = start + i * period * 60
        arrival_time = arrival_time + list(start_time + np.array(cumulativeSum(distances))/ speed * 60 * 60)
        trip_id = trip_id + [i + 1 for _ in range(len(stations))]
        stop_id = stop_id + list(stations.index)
        stop_sequence = stop_sequence + [k + 1 for k in range(len(stations))]

    trip_id = trip_id + list(num_of_trips + np.array(trip_id))
    stop_sequence = stop_sequence + stop_sequence

    reversed_distances = [0] + distances[::-1][:-1]
    for i in range(num_of_trips):
        start_time = start + i * period * 60
        arrival_time = arrival_time + list(start_time + np.array(cumulativeSum(reversed_distances))/ speed * 60 * 60)
        stop_id = stop_id + list(stations.index)[::-1]

    stop_times['trip_id'] = trip_id
    stop_times['arrival_time'] = arrival_time
    stop_times['arrival_time'] = stop_times['arrival_time'].apply(to_time_str)
    stop_times['departure_time'] = stop_times['arrival_time']
    stop_times['stop_id'] = stop_id
    stop_times['stop_sequence'] = stop_sequence


    folder = f'{gtfs_folder}/'
    os.makedirs(folder, exist_ok=True)

    shapes.to_csv(f"{folder}shapes.txt", index = False)
    stops.to_csv(f"{folder}stops.txt", index = False)
    trips.to_csv(f"{folder}trips.txt", index = False)
    stop_times.to_csv(f"{folder}stop_times.txt", index = False)

In [23]:
add_red_line("transitland_copy", 8, 20, 7, 10)

In [26]:
import zipfile
import os

#Zipping up redline data for travel time matrix use
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)  # Get the relative path within the folder
                zipf.write(file_path, arcname)

folder_to_zip = "/transitland_copy"
output_zip_path = "redline.zip"

zip_folder(folder_to_zip, output_zip_path)
print(f"Folder '{folder_to_zip}' zipped to '{output_zip_path}'.")


Folder '/transitland_copy' zipped to 'redline.zip'.
